<a href="https://colab.research.google.com/github/Nourshaolin/Real-Time-Retail-Analytics-and-Product-Recommender/blob/main/RealTime_Retail_Analytics_and_Product_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Real-Time Retail Analytics and Product Recommender
Team members:
* Nour El Moujahid
* Hiba Jebbar
* Ilyass Louzali
* Aya Boumelha

##Milestone 1 (Nour):

Neural Network Workflow (PyTorch): Train deep learning models to predict
purchasing behavior and customer retention based on session logs and historical purchase
data.


###Part 1: Load Kaggle dataset into a Pandas dataframe
Kaggle dataset site: https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset

In [2]:
# Opendatasets is a Python library for downloading datasets from online sources like Kaggle
# Install the library
!pip install opendatasets --upgrade

In [3]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score
import opendatasets as od
import pandas as pd
import torch
import torch.nn as nn

In [4]:
# Download the dataset
# API TOKEN = {"username":"nourelmoujahid","key":"611189b7ac2a80012221c170415923b6"}
data_url = "https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset"
dataset = od.download(data_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nourelmoujahid
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset


In [17]:
# Convert the downloaded dataset into a dataframe
file = 'online-retail-dataset/online_retail_II.xlsx'
df = pd.read_excel(file)

In [6]:
# Pandas dataframe loaded successfully
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


###Part 2: Data Analysis

Features' Information:

* `InvoiceNo`: Invoice number. If this code starts with the letter 'c', it indicates a cancellation.

* `StockCode`: Product (item) code. A 5-digit integral number uniquely assigned to each distinct product.

* `Description`: Product (item) name.

* `Quantity`: The quantities of each product (item) per transaction.

* `InvoiceDate`: The day and time when a transaction was generated.

* `UnitPrice`: Product price per unit in sterling (Â£).

* `CustomerID`: Customer number. A 5-digit integral number uniquely assigned to each customer.

* `Country`: The name of the country where a customer resides.

In [ ]:
#features' data types
df.dtypes

,0
Invoice,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,datetime64[ns]
Price,float64
Customer ID,float64
Country,object


In [ ]:
# 525461 rows
# 8 columns
df.shape

(525461, 8)

In [ ]:
# Available countries
df["Country"].unique()

array(['United Kingdom', 'France', 'USA', 'Belgium', 'Australia', 'EIRE',
       'Germany', 'Portugal', 'Japan', 'Denmark', 'Nigeria',
       'Netherlands', 'Poland', 'Spain', 'Channel Islands', 'Italy',
       'Cyprus', 'Greece', 'Norway', 'Austria', 'Sweden',
       'United Arab Emirates', 'Finland', 'Switzerland', 'Unspecified',
       'Malta', 'Bahrain', 'RSA', 'Bermuda', 'Hong Kong', 'Singapore',
       'Thailand', 'Israel', 'Lithuania', 'West Indies', 'Lebanon',
       'Korea', 'Brazil', 'Canada', 'Iceland'], dtype=object)

In [20]:
# Look for missing values
'''
Description: 2928 NaN values
Customer ID: 107927 NaN values -> relevant
'''
df.isna().sum()

,0
Invoice,0
StockCode,0
Description,2928
Quantity,0
InvoiceDate,0
Price,0
Customer ID,107927
Country,0


#### Part 3: Data processing

In [25]:
# Remove missing value
"""
Notice that Description has no NaN values. I assume that all NaN Description have NaN Customer ID
"""
df.dropna(subset=['Customer ID'], inplace=True)
df.isna().sum()

,0
Invoice,0
StockCode,0
Description,0
Quantity,0
InvoiceDate,0
Price,0
Customer ID,0
Country,0


In [27]:
# Add a 'TotalPrice' column
'''
I sorted just to see who made the biggest purchases
???? = ARE 'M' MISSING VALUES??
'''
df['TotalPrice'] = df['Quantity'] * df['Price']
df.sort_values(by=['TotalPrice'], ascending=False, inplace=True)
df.head(15)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
432176,530715,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,9360,2010-11-04 11:36:00,1.69,15838.0,United Kingdom,15818.40
135013,502263,M,Manual,1,2010-03-23 15:22:00,10953.50,12918.0,United Kingdom,10953.50
358639,524159,M,Manual,1,2010-09-27 16:12:00,10468.80,14063.0,United Kingdom,10468.80
74356,496115,M,Manual,1,2010-01-29 11:04:00,8985.60,17949.0,United Kingdom,8985.60
228042,511465,15044A,PINK PAPER PARASOL,3500,2010-06-08 12:59:00,2.55,18008.0,United Kingdom,8925.00
129903,501766,M,Manual,1,2010-03-19 11:35:00,6958.17,15760.0,Norway,6958.17
129987,501768,M,Manual,1,2010-03-19 11:45:00,6958.17,15760.0,Norway,6958.17
379875,525968,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,3120,2010-10-08 10:10:00,1.66,15838.0,United Kingdom,5179.20
358821,524181,21622,VINTAGE UNION JACK CUSHION COVER,648,2010-09-27 16:59:00,6.89,17450.0,United Kingdom,4464.72
452200,532358,84879,ASSORTED COLOUR BIRD ORNAMENT,2880,2010-11-11 17:05:00,1.45,12931.0,United Kingdom,4176.00


In [29]:
# Group by session (InvoiceNo)

session_data = df.groupby('Invoice').agg(
    {
      'Customer ID': 'first',
      'TotalPrice': 'sum',
      'StockCode': 'nunique',
      'Quantity': 'sum',
      'InvoiceDate': 'first'
    }
  ).reset_index()
session_data

,Invoice,Customer ID,TotalPrice,StockCode,Quantity,InvoiceDate
0,489434,13085.0,505.30,8,166,2009-12-01 07:45:00
1,489435,13085.0,145.80,4,60,2009-12-01 07:46:00
2,489436,13078.0,630.33,19,193,2009-12-01 09:06:00
3,489437,15362.0,310.75,23,145,2009-12-01 09:08:00
4,489438,18102.0,2286.24,17,826,2009-12-01 09:24:00
...,...,...,...,...,...,...
23582,C538121,15535.0,-12.75,1,-1,2010-12-09 15:36:00
23583,C538122,14696.0,-1.25,1,-1,2010-12-09 15:38:00
23584,C538123,12605.0,-7.50,3,-4,2010-12-09 15:41:00
23585,C538124,15329.0,-17.70,3,-6,2010-12-09 15:43:00


In [30]:
# Create binary target: 1 if TotalPrice > 0, else 0
session_data['Purchased'] = (session_data['TotalPrice'] > 0).astype(int)
session_data

,Invoice,Customer ID,TotalPrice,StockCode,Quantity,InvoiceDate,Purchased
0,489434,13085.0,505.30,8,166,2009-12-01 07:45:00,1
1,489435,13085.0,145.80,4,60,2009-12-01 07:46:00,1
2,489436,13078.0,630.33,19,193,2009-12-01 09:06:00,1
3,489437,15362.0,310.75,23,145,2009-12-01 09:08:00,1
4,489438,18102.0,2286.24,17,826,2009-12-01 09:24:00,1
...,...,...,...,...,...,...,...
23582,C538121,15535.0,-12.75,1,-1,2010-12-09 15:36:00,0
23583,C538122,14696.0,-1.25,1,-1,2010-12-09 15:38:00,0
23584,C538123,12605.0,-7.50,3,-4,2010-12-09 15:41:00,0
23585,C538124,15329.0,-17.70,3,-6,2010-12-09 15:43:00,0


In [31]:
# Feature columns
X = session_data[['TotalPrice', 'StockCode', 'Quantity']]
y = session_data['Purchased']

In [32]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [34]:
class PurchasePredictor(nn.Module):
    def __init__(self):
        super(PurchasePredictor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1),
            nn.Sigmoid()  # for binary output
        )

    def forward(self, x):
        return self.model(x)

In [35]:
model = PurchasePredictor()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [36]:
# Training loop
for epoch in range(10):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch).squeeze()
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 138.4854
Epoch 2, Loss: 45.7270
Epoch 3, Loss: 29.1449
Epoch 4, Loss: 22.6953
Epoch 5, Loss: 18.6776
Epoch 6, Loss: 15.7863
Epoch 7, Loss: 13.6596
Epoch 8, Loss: 11.9735
Epoch 9, Loss: 10.7526
Epoch 10, Loss: 9.5095


In [37]:
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor).squeeze()
    predicted_classes = (predictions > 0.5).int()
    accuracy = accuracy_score(y_test_tensor.int(), predicted_classes)
    print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9936


##Milestone 2 (Hiba):
Similarity Search: Use high-dimensional similarity search to identify
customers with comparable behavior, enabling collaborative trends and tailored
suggestions.

### Data Preparation





In [ ]:
# Inspect the data structure first
print(f"Dataset shape: {df.shape}")

Dataset shape: (525461, 8)


In [ ]:
print("\nColumn names:")
print(df.columns.tolist())


Column names:
['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


In [ ]:
print("\nMissing values per column:")
print(df.isnull().sum())


Missing values per column:
Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64


In [ ]:
# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
# Extract date components for time pattern analysis
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek  # 0=Monday, 6=Sunday
df['Hour'] = df['InvoiceDate'].dt.hour

In [ ]:
# Filter out returns (negative quantities) and missing customer IDs
df_clean = df[(df['Quantity'] > 0) & (~df['Customer ID'].isna())]

In [ ]:
print(df_clean[['Quantity', 'Price']].head())
print(df_clean[['Quantity', 'Price']].dtypes)


   Quantity  Price
0        12   6.95
1        12   6.75
2        12   6.75
3        48   2.10
4        24   1.25
Quantity      int64
Price       float64
dtype: object


In [ ]:
df_clean = df[(df['Quantity'] > 0) & (~df['Customer ID'].isna())].copy()
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['Price']


In [ ]:
# Set the latest date in the dataset for recency calculation
latest_date = df_clean['InvoiceDate'].max()

In [ ]:
print(f"\nData ranges from {df_clean['InvoiceDate'].min()} to {latest_date}")
print(f"\nAfter cleaning: {df_clean.shape} (rows, columns)")


Data ranges from 2009-12-01 07:45:00 to 2010-12-09 20:01:00

After cleaning: (407695, 14) (rows, columns)


In [ ]:
# Show the cleaned data
df_clean.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Year,Month,Day,DayOfWeek,Hour,TotalAmount
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009,12,1,1,7,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,7,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,7,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009,12,1,1,7,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009,12,1,1,7,30.0



### 1. Feature Set 1: RFM (Recency, Frequency, Monetary) Metrics




Calculate RFM (Recency, Frequency, Monetary) metrics

RFM helps identify a customer's value based on:
- Recency: How recently they purchased (lower days = more engaged)
- Frequency: How often they purchase (higher = more loyal)bold text
- Monetary: How much they spend (higher = more valuable)


In [ ]:
print("\nColumn names:")
print(df.columns.tolist())


Column names:
['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country', 'Year', 'Month', 'Day', 'DayOfWeek', 'Hour']


In [ ]:
print(list(df_clean.columns))


['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country', 'Year', 'Month', 'Day', 'DayOfWeek', 'Hour', 'TotalAmount']


In [ ]:
def calculate_rfm_metrics(df, latest_date):
    """
    Calculates RFM metrics for each customer.

    Purpose:
    - Recency identifies which customers are currently engaged
    - Frequency identifies loyal customers
    - Monetary identifies high-value customers
    - Average order value shows spending habits per transaction
    """
    # Group by customer on the cleaned dataset
    customer_group = df.groupby('Customer ID')

    # Recency = Days since last purchase
    recency = (latest_date - customer_group['InvoiceDate'].max()).dt.days.reset_index()
    recency.columns = ['Customer ID', 'Recency']

    # Frequency = Number of unique purchases
    frequency = customer_group['Invoice'].nunique().reset_index()
    frequency.columns = ['Customer ID', 'Frequency']

    # Monetary = Total spent
    monetary = customer_group['TotalAmount'].sum().reset_index()
    monetary.columns = ['Customer ID', 'Monetary']

    # Average order value
    avg_order = customer_group['TotalAmount'].mean().reset_index()
    avg_order.columns = ['Customer ID', 'AvgOrderValue']

    # Merge all metrics
    rfm = recency.merge(frequency, on='Customer ID')\
                 .merge(monetary, on='Customer ID')\
                 .merge(avg_order, on='Customer ID')

    return rfm
# Calculate RFM metrics
rfm_features = calculate_rfm_metrics(df_clean, latest_date)

# Display the RFM metrics for a few customers
print(f"RFM metrics calculated for {len(rfm_features)} customers")
rfm_features.head()

RFM metrics calculated for 4314 customers


,Customer ID,Recency,Frequency,Monetary,AvgOrderValue
0,12346.0,164,11,372.86,11.298788
1,12347.0,2,2,1323.32,18.638310
2,12348.0,73,1,222.16,11.108000
3,12349.0,42,3,2671.14,26.187647
4,12351.0,10,1,300.93,14.330000


### 2.   Feature Set 2: Time Patterns (Day/Hour of Purchases)









In [ ]:
def calculate_time_patterns(df):
    """
    Extracts time-based purchase patterns for each customer.
    """
    # Create day of week dummies (0=Monday, 6=Sunday)
    dow_dummies = pd.get_dummies(df['DayOfWeek'], prefix='DayOfWeek')
    df_with_dow = pd.concat([df[['Customer ID', 'Invoice']], dow_dummies], axis=1)

    # Calculate average day of week distribution per customer
    dow_dist = df_with_dow.groupby('Customer ID')[dow_dummies.columns].mean().reset_index()

    # Group hours into time periods
    df['TimeOfDay'] = pd.cut(df['Hour'],
                             bins=[0, 6, 12, 18, 24],
                             labels=['Night', 'Morning', 'Afternoon', 'Evening'],
                             right=False)

    # Create time of day dummies
    tod_dummies = pd.get_dummies(df['TimeOfDay'], prefix='TimeOfDay')
    df_with_tod = pd.concat([df[['Customer ID', 'Invoice']], tod_dummies], axis=1)

    # Calculate average time of day distribution per customer
    tod_dist = df_with_tod.groupby('Customer ID')[tod_dummies.columns].mean().reset_index()

    # Create month dummies
    month_dummies = pd.get_dummies(df['Month'], prefix='Month')
    df_with_month = pd.concat([df[['Customer ID', 'Invoice']], month_dummies], axis=1)

    # Calculate average monthly distribution per customer
    month_dist = df_with_month.groupby('Customer ID')[month_dummies.columns].mean().reset_index()

    # Combine all time patterns
    time_patterns = dow_dist.merge(tod_dist, on='Customer ID')\
                            .merge(month_dist, on='Customer ID')

    return time_patterns

# Calculate time patterns
time_pattern_features = calculate_time_patterns(df_clean)
# Display the time patterns for a few customers
print(f"Time patterns extracted with {time_pattern_features.shape[1] - 1} features")
time_pattern_features.head()


Time patterns extracted with 23 features


,Customer ID,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,DayOfWeek_5,DayOfWeek_6,TimeOfDay_Night,TimeOfDay_Morning,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
0,12346.0,0.727273,0.151515,0.0,0.030303,0.090909,0.0,0.00000,0.0,0.212121,...,0.151515,0.00000,0.000000,0.575758,0.0,0.0,0.0,0.000000,0.0,0.151515
1,12347.0,0.000000,0.436620,0.0,0.000000,0.000000,0.0,0.56338,0.0,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.563380,0.0,0.436620
2,12348.0,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.0,0.0,1.0,0.000000,0.0,0.000000
3,12349.0,0.000000,0.009804,0.0,0.990196,0.000000,0.0,0.00000,0.0,0.549020,...,0.000000,0.45098,0.009804,0.000000,0.0,0.0,0.0,0.539216,0.0,0.000000
4,12351.0,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.0,0.000000


###3. Feature Set 3: Product Category Preferences

In [ ]:
def calculate_category_preferences(df):
    """
    Analyzes product category preferences for each customer.
    """
    # Extract product categories (first word of Description as a simple approach)
    df['Category'] = df['Description'].str.split().str[0]

    # Get top 20 categories by frequency
    top_categories = df['Category'].value_counts().nlargest(20).index

    # Filter only rows with top categories
    category_data = df[df['Category'].isin(top_categories)]

    # One-hot encode categories per customer
    category_pivot = pd.crosstab(category_data['Customer ID'], category_data['Category'])

    # Rename columns to have a consistent prefix
    category_pivot.columns = [f'Cat_{col}' for col in category_pivot.columns]

    # Calculate category diversity (number of unique categories purchased)
    cat_diversity = df.groupby('Customer ID')['Category'].nunique().reset_index()
    cat_diversity.columns = ['Customer ID', 'CategoryDiversity']

    # Combine category preferences
    category_prefs = category_pivot.reset_index().merge(cat_diversity, on='Customer ID')

    return category_prefs

# Calculate category preferences
category_preference_features = calculate_category_preferences(df_clean)
# Display the category preferences for a few customers
print(f"Category preferences extracted with {category_preference_features.shape[1] - 1} features")
category_preference_features.head()


Category preferences extracted with 21 features


,Customer ID,Cat_ASSORTED,Cat_BLUE,Cat_CHRISTMAS,Cat_DOOR,Cat_FELTCRAFT,Cat_HEART,Cat_JUMBO,Cat_LUNCH,Cat_PACK,...,Cat_RED,Cat_RETRO,Cat_SET,Cat_SMALL,Cat_STRAWBERRY,Cat_TEA,Cat_VINTAGE,Cat_WHITE,Cat_WOODEN,CategoryDiversity
0,12346.0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,4
1,12347.0,0,3,0,0,0,1,1,0,1,...,3,0,2,1,0,1,0,1,0,47
2,12348.0,0,0,0,0,0,0,0,0,8,...,0,0,3,0,0,0,0,0,0,9
3,12349.0,1,0,0,5,0,1,0,1,1,...,11,2,3,0,2,0,1,1,0,53
4,12351.0,0,1,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,16


###4. Feature Set 4: Purchase Variability and Basket Analysis

    Calculates metrics related to purchase patterns and basket characteristics.
    
    Purpose:
    - Measures consistency in shopping behavior
    - Analyzes shopping basket composition
    - Identifies price sensitivity
    

In [ ]:
import numpy as np


In [ ]:
# Calculate metrics related to purchase variability and basket characteristics
# This reveals shopping consistency and basket behaviors

def calculate_purchase_variability(df):
    """
    Calculates metrics related to purchase patterns and basket characteristics.

    Purpose:
    - Measures consistency in shopping behavior
    - Analyzes shopping basket composition
    - Identifies price sensitivity
    """
    # Calculate average basket size (items per transaction)
    basket_items = df.groupby(['Customer ID', 'Invoice'])['Quantity'].sum().reset_index()
    avg_basket = basket_items.groupby('Customer ID')['Quantity'].mean().reset_index()
    avg_basket.columns = ['Customer ID', 'AvgBasketSize']

    # Calculate basket size variability
    basket_var = basket_items.groupby('Customer ID')['Quantity'].std().reset_index()
    basket_var.columns = ['Customer ID', 'BasketSizeVariability']
    basket_var = basket_var.fillna(0)  # Fill NaN for customers with only one purchase

    # Calculate price sensitivity (average unit price paid)
    price_sensitivity = df.groupby('Customer ID')['Price'].mean().reset_index()
    price_sensitivity.columns = ['Customer ID', 'AvgUnitPrice']

    # Calculate purchase intervals
    # First get the first purchase date per invoice for each customer
    invoice_dates = df.groupby(['Customer ID', 'Invoice'])['InvoiceDate'].min().reset_index()
    invoice_dates = invoice_dates.sort_values(['Customer ID', 'InvoiceDate'])

    # Function to calculate intervals between purchases
    def get_purchase_intervals(group):
        if len(group) <= 1:
            return 0, 0  # No interval for single purchase

        # Calculate days between consecutive purchases
        group = group.sort_values('InvoiceDate')
        intervals = group['InvoiceDate'].diff().dt.days.dropna().tolist()

        if not intervals:
            return 0, 0

        return np.mean(intervals), np.std(intervals) if len(intervals) > 1 else 0

    # Apply the function to each customer group
    purchase_intervals = []
    for customer_id, group in invoice_dates.groupby('Customer ID'):
        mean_interval, std_interval = get_purchase_intervals(group)
        purchase_intervals.append({
            'Customer ID': customer_id,
            'PurchaseInterval': mean_interval,
            'PurchaseIntervalStd': std_interval
        })

    purchase_intervals_df = pd.DataFrame(purchase_intervals)

    # Combine all variability metrics
    purchase_variability = avg_basket.merge(basket_var, on='Customer ID', how='left')\
                                    .merge(price_sensitivity, on='Customer ID', how='left')\
                                    .merge(purchase_intervals_df, on='Customer ID', how='left')

    # Fill NaN values
    purchase_variability = purchase_variability.fillna(0)

    return purchase_variability

# Calculate purchase variability metrics
purchase_variability_features = calculate_purchase_variability(df_clean)

# Display the purchase variability for a few customers
print(f"Purchase variability features calculated")
purchase_variability_features.head()

Purchase variability features calculated


,Customer ID,AvgBasketSize,BasketSizeVariability,AvgUnitPrice,PurchaseInterval,PurchaseIntervalStd
0,12346.0,6.363636,4.653444,6.253333,19.2,34.778729
1,12347.0,414.000000,134.350288,2.295070,37.0,0.000000
2,12348.0,373.000000,0.000000,0.719500,0.0,0.000000
3,12349.0,331.000000,286.712399,8.581765,90.0,72.000000
4,12351.0,261.000000,0.000000,2.355238,0.0,0.000000


###5. Feature Set 5: Customer Longevity

    Calculates customer longevity-related metrics.
    
    Purpose:
    - Identifies customer lifecycle stage (new vs. established)
    - Measures customer retention timespan


In [ ]:
def calculate_customer_longevity(df):
    """
    Calculates the number of days between a customer's first and last purchase.
    """
    # Find first and last purchase dates based on InvoiceDate
    first_purchase = df.groupby('Customer ID')['InvoiceDate'].min().reset_index()
    first_purchase.columns = ['Customer ID', 'FirstPurchase']

    last_purchase = df.groupby('Customer ID')['InvoiceDate'].max().reset_index()
    last_purchase.columns = ['Customer ID', 'LastPurchase']

    # Merge and calculate lifespan
    dates = first_purchase.merge(last_purchase, on='Customer ID')
    dates['CustomerLifespan'] = (dates['LastPurchase'] - dates['FirstPurchase']).dt.days

    return dates[['Customer ID', 'CustomerLifespan']]
# Calculate longevity metrics
longevity_features = calculate_customer_longevity(df_clean)

# Display result
print(f"Customer longevity calculated for {len(longevity_features)} customers")
longevity_features.head()


Customer longevity calculated for 4314 customers


,Customer ID,CustomerLifespan
0,12346.0,196
1,12347.0,37
2,12348.0,0
3,12349.0,181
4,12351.0,0


### Combine All Feature Sets and Normalize

In [ ]:
'rfm_features' in locals()


True

In [ ]:
# Combine all the feature sets into customer vectors
# This creates the final representation for similarity search

# Merge all feature sets
customer_features = rfm_features.merge(time_pattern_features, on='Customer ID', how='left')\
                      .merge(category_preference_features, on='Customer ID', how='left')\
                      .merge(purchase_variability_features, on='Customer ID', how='left')\
                      .merge(longevity_features, on='Customer ID', how='left')

# Fill missing values with appropriate defaults
customer_features = customer_features.fillna(0)

# Display the final feature set dimensions
print(f"Created feature vectors for {len(customer_features)} customers with {customer_features.shape[1] - 1} features")
print(f"Feature columns: {customer_features.columns.tolist()}")
customer_features.head()

Created feature vectors for 4314 customers with 54 features
Feature columns: ['Customer ID', 'Recency', 'Frequency', 'Monetary', 'AvgOrderValue', 'DayOfWeek_0', 'DayOfWeek_1', 'DayOfWeek_2', 'DayOfWeek_3', 'DayOfWeek_4', 'DayOfWeek_5', 'DayOfWeek_6', 'TimeOfDay_Night', 'TimeOfDay_Morning', 'TimeOfDay_Afternoon', 'TimeOfDay_Evening', 'Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12', 'Cat_ASSORTED', 'Cat_BLUE', 'Cat_CHRISTMAS', 'Cat_DOOR', 'Cat_FELTCRAFT', 'Cat_HEART', 'Cat_JUMBO', 'Cat_LUNCH', 'Cat_PACK', 'Cat_PAPER', 'Cat_PINK', 'Cat_RED', 'Cat_RETRO', 'Cat_SET', 'Cat_SMALL', 'Cat_STRAWBERRY', 'Cat_TEA', 'Cat_VINTAGE', 'Cat_WHITE', 'Cat_WOODEN', 'CategoryDiversity', 'AvgBasketSize', 'BasketSizeVariability', 'AvgUnitPrice', 'PurchaseInterval', 'PurchaseIntervalStd', 'CustomerLifespan']


,Customer ID,Recency,Frequency,Monetary,AvgOrderValue,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4,...,Cat_VINTAGE,Cat_WHITE,Cat_WOODEN,CategoryDiversity,AvgBasketSize,BasketSizeVariability,AvgUnitPrice,PurchaseInterval,PurchaseIntervalStd,CustomerLifespan
0,12346.0,164,11,372.86,11.298788,0.727273,0.151515,0.0,0.030303,0.090909,...,0.0,0.0,0.0,4.0,6.363636,4.653444,6.253333,19.2,34.778729,196
1,12347.0,2,2,1323.32,18.638310,0.000000,0.436620,0.0,0.000000,0.000000,...,0.0,1.0,0.0,47.0,414.000000,134.350288,2.295070,37.0,0.000000,37
2,12348.0,73,1,222.16,11.108000,1.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,9.0,373.000000,0.000000,0.719500,0.0,0.000000,0
3,12349.0,42,3,2671.14,26.187647,0.000000,0.009804,0.0,0.990196,0.000000,...,1.0,1.0,0.0,53.0,331.000000,286.712399,8.581765,90.0,72.000000,181
4,12351.0,10,1,300.93,14.330000,1.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,16.0,261.000000,0.000000,2.355238,0.0,0.000000,0


### Normalize Features for Similarity Search

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# Normalize the features to ensure equal contribution in similarity calculations
# This step is critical for accurate similarity search results

# Separate customer IDs from features
customer_ids = customer_features['Customer ID']
feature_columns = customer_features.columns.drop('Customer ID')

# Apply standardization (mean=0, std=1)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_features[feature_columns])

# Create a DataFrame with the normalized features
normalized_features = pd.DataFrame(scaled_features, columns=feature_columns)
normalized_features.insert(0, 'CustomerID', customer_ids)

print("Features have been normalized. Ready for similarity search!")
normalized_features.head()

# Optionally save the normalized features for later use
normalized_features.to_csv('customer_feature_vectors.csv', index=False)

Features have been normalized. Ready for similarity search!


### Implementing FAISS for Similarity Search

In [ ]:
!pip install faiss-cpu

In [ ]:
# Implement similarity search using FAISS with cosine similarity 1111
# This allows efficient finding of similar customers
import faiss
import numpy as np

# Extract just the feature columns (excluding CustomerID)
feature_columns = normalized_features.columns.drop('CustomerID')
features_array = normalized_features[feature_columns].values.astype('float32')

# Normalize vectors to unit length for cosine similarity
features_array_norm = features_array.copy()
faiss.normalize_L2(features_array_norm)  # In-place normalization

# Create a FAISS index with inner product (for cosine similarity)
dimension = features_array_norm.shape[1]  # Dimensionality of our feature vectors
index = faiss.IndexFlatIP(dimension)  # Inner Product for cosine similarity
index.add(features_array_norm)  # Add all normalized customer vectors to the index

# Function to find similar customers for a given customer ID
def find_similar_customers(customer_id, k=5):
    """
    Finds k most similar customers to the given customer ID using cosine similarity.

    Parameters:
    customer_id: ID of the target customer
    k: Number of similar customers to return

    Returns:
    List of tuples (customer_id, similarity_score)
    """
    # Get the customer index in our arrays
    customer_idx = normalized_features[normalized_features['CustomerID'] == customer_id].index

    if len(customer_idx) == 0:
        return f"Customer ID {customer_id} not found"

    customer_idx = customer_idx[0]

    # Get the feature vector for this customer and normalize it
    query_vector = features_array[customer_idx].reshape(1, -1).copy()
    faiss.normalize_L2(query_vector)  # Normalize the query vector

    # Search for k+1 most similar customers (including self)
    similarities, indices = index.search(query_vector, k+1)

    # Remove the query customer from results (it will be the first match)
    similar_indices = indices[0][1:]
    similarity_scores = similarities[0][1:]  # These are already similarity scores (not distances)

    # Get customer IDs
    similar_customers = normalized_features.iloc[similar_indices]['CustomerID'].values

    return list(zip(similar_customers, similarity_scores))

# Example: Find similar customers for a specific customer ID
sample_customer_id = 17850
similar_customers = find_similar_customers(sample_customer_id, k=5)
print(f"Customers similar to {sample_customer_id}:")
for customer_id, similarity in similar_customers:
    print(f"Customer {customer_id}: Similarity score {similarity:.4f}")

Customers similar to 17850:
Customer 16019.0: Similarity score 0.7483
Customer 16814.0: Similarity score 0.6858
Customer 16713.0: Similarity score 0.6807
Customer 17377.0: Similarity score 0.6556
Customer 16525.0: Similarity score 0.6554


### Preprocess & Vector Setup

In [ ]:
# Filter out rows with missing CustomerID
df = df[df['Customer ID'].notnull()]
df['Customer ID'] = df['Customer ID'].astype(int)

# Make sure features are normalized (if not done already, normalize before this step)
customer_ids = normalized_features.index.to_numpy()
vectors = normalized_features.to_numpy().astype('float32')  # faiss needs float32


### It takes a customer ID as input and return a list of similar customer IDs, or a message if the ID is not available.

In [ ]:
import faiss
import numpy as np
import pandas as pd

# ... (your existing code to prepare df, normalized_features, etc.) ...

# Make sure features are normalized (if not done already, normalize before this step)
customer_ids = normalized_features['CustomerID'].to_numpy()
feature_columns = normalized_features.columns.drop('CustomerID')
vectors = normalized_features[feature_columns].to_numpy().astype('float32')

# Build FAISS index with L2 distance
index = faiss.IndexFlatL2(vectors.shape[1])
index.add(vectors)

def find_similar_customers(customer_id, k=5):
    """
    Finds k most similar customers to the given customer ID.

    Parameters:
    customer_id: ID of the target customer
    k: Number of similar customers to return

    Returns:
    List of similar customer IDs or a message if the customer is not found.
    """

    if customer_id not in normalized_features['CustomerID'].values:
        return "Customer not available."

    # Get the index of the target customer in the feature vectors
    customer_index = normalized_features[normalized_features['CustomerID'] == customer_id].index[0]

    # Get the feature vector for the target customer
    query_vector = vectors[customer_index].reshape(1, -1)

    # Search for similar customers using FAISS
    distances, indices = index.search(query_vector, k + 1)  # +1 to include the target customer

    # Get the similar customer IDs (excluding the target customer)
    similar_customer_ids = [int(customer_ids[i]) for i in indices[0][1:]]

    return similar_customer_ids

# Get customer ID input from the user
customer_id_input = int(input("Enter a customer ID: "))

# Find similar customers
similar_customers = find_similar_customers(customer_id_input)

# Print the results
print(similar_customers)



Enter a customer ID: 17850
[17377, 13089, 15311, 12748, 15039]


### KMeans + Cluster Labeling

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
cluster_labels = kmeans.fit_predict(vectors)

# Create a lookup dictionary: customer_id -> cluster label
cluster_lookup = {cid: int(label) for cid, label in zip(customer_ids, cluster_labels)}


### Recommendation per Cluster

This output is telling you the top 5 most popular products (by total quantity purchased) for each customer cluster. Essentially, what types of products each group of similar customers is most interested in.



In [ ]:
trending_products = {}

for cluster_id in range(kmeans.n_clusters):
    cluster_customers = [cid for cid, label in cluster_lookup.items() if label == cluster_id]
    # Change 'CustomerID' to 'Customer ID'
    cluster_df = df[df['Customer ID'].isin(cluster_customers)]

    # Remove missing descriptions
    cluster_df = cluster_df[cluster_df['Description'].notnull()]

    top_products = (
        cluster_df.groupby('Description')['Quantity']
        .sum()
        .sort_values(ascending=False)
        .head(5)
    )

    trending_products[cluster_id] = top_products
    print(f"\nCluster {cluster_id} Trending Products:")
    print(top_products)


Cluster 0 Trending Products:
Description
WHITE HANGING HEART T-LIGHT HOLDER     5588
COLOUR GLASS T-LIGHT HOLDER HANGING    5444
MINI HIGHLIGHTER PENS                  4560
ASSTD DESIGN BUBBLE GUM RING           3780
PACK OF 72 RETRO SPOT CAKE CASES       3520
Name: Quantity, dtype: int64

Cluster 1 Trending Products:
Description
ASSORTED COLOUR BIRD ORNAMENT         15033
WHITE HANGING HEART T-LIGHT HOLDER    14504
PACK OF 72 RETRO SPOT CAKE CASES      13546
BROCADE RING PURSE                    11713
JUMBO BAG RED RETROSPOT                9804
Name: Quantity, dtype: int64

Cluster 2 Trending Products:
Description
WORLD WAR 2 GLIDERS ASSTD DESIGNS     42745
BROCADE RING PURSE                    33388
WHITE HANGING HEART T-LIGHT HOLDER    28279
BLACK AND WHITE PAISLEY FLOWER MUG    25258
ASSORTED COLOUR BIRD ORNAMENT         24051
Name: Quantity, dtype: int64

Cluster 3 Trending Products:
Description
SET/6 FRUIT SALAD PAPER CUPS          7131
SET/6 FRUIT SALAD  PAPER PLATES       7009

## Milestone 4 (Aya)
Large-Scale Hypothesis Testing: Conduct hypothesis testing across patient
subgroups to uncover statistically significant health factors. Apply corrections for multiple
testing (e.g., Benjamini-Hochberg).

##Milestone 5 (Louzali)
Recommendation Systems: Develop a recommendation engine to propose
lifestyle changes and follow-up actions based on predicted risks and matched patient
cohorts.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix # For sparse user-item matrix
import os # Added for listing directories

In [ ]:
def process_loaded_dataframes(df_sheet1: pd.DataFrame, df_sheet2: pd.DataFrame):
    """Combines and preprocesses two pre-loaded pandas DataFrames."""
    print("Processing pre-loaded DataFrames...")
    if df_sheet1 is None or df_sheet2 is None:
        print("Error: One or both input DataFrames are None. Cannot proceed with processing.")
        return None

    if not isinstance(df_sheet1, pd.DataFrame) or not isinstance(df_sheet2, pd.DataFrame):
        print("Error: Inputs must be pandas DataFrames.")
        return None

    # Concatenate the data from both DataFrames
    df = pd.concat([df_sheet1, df_sheet2], ignore_index=True)
    print(f"Initial rows after concatenating input DataFrames: {len(df)}")

    # --- Start of common preprocessing steps ---
    # Drop rows with missing CustomerID or Description
    # Check if required columns exist
    required_cols = ['Customer ID', 'Description', 'Quantity', 'StockCode']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        print(f"Error: The combined DataFrame is missing required columns: {', '.join(missing_cols)}")
        return None

    df.dropna(subset=['Customer ID', 'Description'], inplace=True)
    # Ensure CustomerID is integer
    df['Customer ID'] = df['Customer ID'].astype(int)
    # Filter out transactions with non-positive quantity
    df = df[df['Quantity'] > 0]

    # Filter out non-product StockCodes (e.g., postage, manual adjustments)
    non_product_codes = ['POST', 'D', 'M', 'BANK CHARGES', 'S', 'AMAZONFEE', 'CRUK', 'DOT', 'MANUAL', 'PADS']
    df = df[~df['StockCode'].astype(str).isin(non_product_codes)]

    # Filter based on description keywords that indicate non-product entries
    non_product_description_keywords = ['POSTAGE', 'MANUAL', 'CHARGES', 'FEES', 'CARRIAGE', 'DOTCOM', 'CRUK']
    pattern = '|'.join(non_product_description_keywords)
    # Ensure Description is string type before using .str.contains
    df['Description'] = df['Description'].astype(str)
    df = df[~df['Description'].str.contains(pattern, case=False, na=False, regex=True)]

    # Clean Description: remove leading/trailing spaces and convert to lowercase
    df['Description'] = df['Description'].str.strip().str.lower()
    # Clean StockCode: convert to string and remove leading/trailing spaces
    df['StockCode'] = df['StockCode'].astype(str).str.strip()

    # Create a 'LogQuantity' column to use as rating (smoothes high quantities and handles 0)
    df['LogQuantity'] = np.log1p(df['Quantity'])

    # Select relevant columns for the cleaned DataFrame
    df_cleaned = df[['Customer ID', 'StockCode', 'Description', 'LogQuantity', 'Quantity']].copy()

    # Aggregate interactions: if a user bought the same item multiple times (possibly in different transactions),
    # group them and take the first description, max LogQuantity, and sum of Quantity.
    df_cleaned = df_cleaned.groupby(['Customer ID', 'StockCode']).agg({
        'Description': 'first',
        'LogQuantity': 'max',
        'Quantity': 'sum'
    }).reset_index()
    print(f"Rows after grouping by CustomerID-StockCode: {len(df_cleaned)}")

    if df_cleaned.empty:
        print("Error: No data remaining after preprocessing. Check filters and data integrity.")
        return None

    print("Data preprocessing complete.")
    return df_cleaned

In [ ]:
def create_content_based_components(df_cleaned):
    """Creates TF-IDF matrix and item similarity matrix for content-based filtering."""
    print("Building content-based components...")
    if df_cleaned is None or df_cleaned.empty:
        print("Error: Cannot build content-based components with no data.")
        return None, None, None, None, None

    item_metadata = df_cleaned[['StockCode', 'Description']].drop_duplicates(subset=['StockCode']).set_index('StockCode')

    if item_metadata.empty or item_metadata['Description'].isnull().all():
        print("Error: No valid item metadata or descriptions for CB.")
        return None, None, None, None, None

    tfidf_vectorizer = TfidfVectorizer(stop_words='english', min_df=2)
    try:
        valid_descriptions = item_metadata['Description'].dropna()
        if valid_descriptions.empty:
            print("Error: No valid descriptions available for TF-IDF vectorization after dropping NaNs.")
            return None, None, None, None, None
        tfidf_matrix = tfidf_vectorizer.fit_transform(valid_descriptions)
        cb_item_stock_codes = valid_descriptions.index.tolist()
        cb_item_index_map = {stock_code: i for i, stock_code in enumerate(cb_item_stock_codes)}
    except ValueError as e:
        print(f"Error during TF-IDF fitting: {e}")
        return None, None, None, None, None
    cb_cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    print("Content-based components built.")
    return cb_cosine_sim_matrix, cb_item_index_map, cb_item_stock_codes, item_metadata, tfidf_vectorizer

In [ ]:
def create_collaborative_filtering_components(df_cleaned):
    """Creates user-item matrix and item-item similarity for collaborative filtering."""
    print("Building collaborative filtering components (Item-Based)...")
    if df_cleaned is None or df_cleaned.empty:
        print("Error: Cannot build CF components with no data.")
        return None, None, None, None
    try:
        user_item_df = df_cleaned.pivot_table(index='Customer ID', columns='StockCode', values='LogQuantity').fillna(0)
    except Exception as e:
        print(f"Error creating user-item matrix: {e}")
        return None, None, None, None
    if user_item_df.empty:
        print("User-item matrix is empty after pivot_table. Check input data for CF.")
        return None, None, None, None
    item_item_cf_similarity_matrix = cosine_similarity(user_item_df.T.values)
    cf_item_stock_codes = user_item_df.columns.tolist()
    cf_item_index_map = {stock_code: i for i, stock_code in enumerate(cf_item_stock_codes)}
    print("Collaborative filtering components built.")
    return user_item_df, item_item_cf_similarity_matrix, cf_item_index_map, cf_item_stock_codes

In [ ]:
def get_hybrid_recommendations(user_id, N_total,
                               user_item_df, item_item_cf_similarity_matrix,
                               cf_item_index_map, cf_item_stock_codes,
                               cb_cosine_sim_matrix, cb_item_index_map,
                               cb_item_stock_codes, item_metadata,
                               df_cleaned, alpha=0.5):
    print(f"\nGenerating hybrid recommendations for User ID: {user_id}...")
    if user_item_df is None or item_item_cf_similarity_matrix is None or cb_cosine_sim_matrix is None or item_metadata is None:
        print("Error: One or more model components are missing for hybrid recommendations.")
        return pd.DataFrame()
    user_purchases_stockcodes = df_cleaned[df_cleaned['Customer ID'] == user_id]['StockCode'].unique().tolist()
    all_known_stock_codes = set(cf_item_stock_codes) | set(cb_item_stock_codes)
    candidate_items = [sc for sc in all_known_stock_codes if sc not in user_purchases_stockcodes]
    if not candidate_items:
        print(f"User {user_id} has purchased all available items, or no candidate items found.")
        return pd.DataFrame()

    cf_scores = {}
    if user_id in user_item_df.index:
        user_ratings_for_cf = user_item_df.loc[user_id]
        for item_code_candidate in candidate_items:
            if item_code_candidate not in cf_item_index_map:
                cf_scores[item_code_candidate] = 0.0
                continue
            candidate_idx_cf = cf_item_index_map[item_code_candidate]
            weighted_sum_sims, sum_of_sims = 0.0, 0.0
            for purchased_item_code, rating in user_ratings_for_cf.items():
                if rating > 0 and purchased_item_code in cf_item_index_map:
                    purchased_idx_cf = cf_item_index_map[purchased_item_code]
                    if candidate_idx_cf < item_item_cf_similarity_matrix.shape[0] and \
                       purchased_idx_cf < item_item_cf_similarity_matrix.shape[1]:
                        similarity = item_item_cf_similarity_matrix[candidate_idx_cf, purchased_idx_cf]
                        if similarity > 0:
                            weighted_sum_sims += similarity * rating
                            sum_of_sims += similarity
            cf_scores[item_code_candidate] = weighted_sum_sims / sum_of_sims if sum_of_sims > 0 else 0.0
    else:
        for item_code_candidate in candidate_items: cf_scores[item_code_candidate] = 0.0

    cb_scores = {}
    if user_purchases_stockcodes:
        for item_code_candidate in candidate_items:
            if item_code_candidate not in cb_item_index_map:
                cb_scores[item_code_candidate] = 0.0
                continue
            candidate_idx_cb = cb_item_index_map[item_code_candidate]
            max_sim_to_profile = 0.0
            for purchased_item_code in user_purchases_stockcodes:
                if purchased_item_code in cb_item_index_map:
                    purchased_idx_cb = cb_item_index_map[purchased_item_code]
                    if candidate_idx_cb < cb_cosine_sim_matrix.shape[0] and \
                       purchased_idx_cb < cb_cosine_sim_matrix.shape[1]:
                        similarity = cb_cosine_sim_matrix[candidate_idx_cb, purchased_idx_cb]
                        if similarity > max_sim_to_profile: max_sim_to_profile = similarity
            cb_scores[item_code_candidate] = max_sim_to_profile
    else:
        for item_code_candidate in candidate_items: cb_scores[item_code_candidate] = 0.0

    hybrid_scores = []
    all_cf_values = [s for s in cf_scores.values() if pd.notnull(s) and s > 0]
    min_cf_score = min(all_cf_values) if all_cf_values else 0
    max_cf_score = max(all_cf_values) if all_cf_values else 1

    def normalize_score(score, min_s, max_s):
        if max_s == min_s: return 0.5 if score > 0 else 0.0
        if pd.isnull(score) or score <= 0: return 0.0
        return (score - min_s) / (max_s - min_s)

    for item_code in candidate_items:
        cf_s = cf_scores.get(item_code, 0.0)
        cb_s = cb_scores.get(item_code, 0.0)
        norm_cf = normalize_score(cf_s, min_cf_score, max_cf_score)
        hybrid_score = alpha * norm_cf + (1 - alpha) * cb_s
        desc = item_metadata.loc[item_code, 'Description'] if item_code in item_metadata.index else "N/A"
        hybrid_scores.append({
            'StockCode': item_code, 'Description': desc, 'CF_Score_Raw': cf_s,
            'CF_Score_Norm': norm_cf, 'CB_Score': cb_s, 'Hybrid_Score': hybrid_score
        })
    recommendations_df = pd.DataFrame(hybrid_scores).sort_values(by='Hybrid_Score', ascending=False)
    return recommendations_df.head(N_total)

In [ ]:
if __name__ == "__main__":
    # Define paths for Kaggle environment
    user_excel_data_source = 'online-retail-dataset/online_retail_II.xlsx'

    # Sheet names within the 'Online Retail.xlsx' file.
    sheet_name_2010_2011 = 'Year 2010-2011'
    sheet_name_2009_2010 = 'Year 2009-2010'

    df_sheet_1 = None
    df_sheet_2 = None
    df_cleaned = None

    try:
        print(f"Attempting to load sheet: '{sheet_name_2010_2011}' from '{full_excel_path}'")
        df_sheet_1 = pd.read_excel(full_excel_path, sheet_name=sheet_name_2010_2011)
        print(f"Successfully loaded sheet: {sheet_name_2010_2011}")

        print(f"Attempting to load sheet: '{sheet_name_2009_2010}' from '{full_excel_path}'")
        df_sheet_2 = pd.read_excel(full_excel_path, sheet_name=sheet_name_2009_2010)
        print(f"Successfully loaded sheet: {sheet_name_2009_2010}")

        # Call the data processing function with the pre-loaded DataFrames
        df_cleaned = process_loaded_dataframes(df_sheet_1, df_sheet_2)

    except FileNotFoundError:
        print(f"Error: Excel file not found at '{full_excel_path}'.")
        print("Please ensure the dataset is correctly added to your Kaggle Notebook and the path is accurate.")
        print("Listing contents of '../input/' directory to help diagnose:")
        try:
            input_contents = os.listdir('../input/')
            if input_contents:
                print("Contents of '../input/':")
                for item in input_contents: print(f"  - {item}")
                expected_dataset_dir = os.path.dirname(full_excel_path)
                if os.path.exists(expected_dataset_dir):
                    print(f"\nContents of '{expected_dataset_dir}':")
                    for item in os.listdir(expected_dataset_dir): print(f"  - {item}")
                else: print(f"\nNote: The directory '{expected_dataset_dir}' itself was not found.")
            else: print("  ('../input/' directory is empty or inaccessible)")
        except Exception as e_listdir: print(f"  (Could not list directory contents due to: {e_listdir})")
    except ValueError as e: # Handles incorrect sheet names
        print(f"Error reading sheets from Excel file: {e}")
        print(f"Please ensure the Excel file '{full_excel_path}' contains sheets named '{sheet_name_2010_2011}' and '{sheet_name_2009_2010}'.")
    except Exception as e:
        print(f"An unexpected error occurred during data loading: {e}")

    if df_cleaned is not None and not df_cleaned.empty:
        print("Dataframes loaded and preprocessed successfully.")
        print(df_cleaned.head())

        print("\n--- Building Recommender System Components ---")
        cb_components = create_content_based_components(df_cleaned)
        if cb_components:
            cb_cosine_sim_matrix, cb_item_index_map, cb_item_stock_codes, item_metadata, tfidf_vectorizer = cb_components
            print("Content-based components successfully created.")
        else:
            print("Failed to create content-based components. Exiting.")
            exit()

        cf_components = create_collaborative_filtering_components(df_cleaned)
        if cf_components:
            user_item_df, item_item_cf_similarity_matrix, cf_item_index_map, cf_item_stock_codes = cf_components
            print("Collaborative filtering components successfully created.")
        else:
            print("Failed to create collaborative filtering components. Exiting.")
            exit()

        if user_item_df is not None and not user_item_df.empty:
            sample_user_id = None
            user_purchase_counts = df_cleaned['Customer ID'].value_counts()
            for uid_candidate in user_purchase_counts.index:
                if uid_candidate in user_item_df.index:
                    sample_user_id = uid_candidate
                    print(f"\nSelected sample user ID (exists in CF matrix & has purchase history): {sample_user_id}")
                    break
            if sample_user_id is None:
                if not user_item_df.empty:
                    sample_user_id = user_item_df.index[0]
                    print(f"\nSelected fallback sample user ID (first user in CF matrix): {sample_user_id}")
                else:
                    print("CF User-item matrix is empty, cannot select a sample user.")

            if sample_user_id is not None:
                print(f"\n--- Recommendations for User ID: {sample_user_id} ---")
                user_history = df_cleaned[df_cleaned['Customer ID'] == sample_user_id].sort_values(by='LogQuantity', ascending=False)
                print("\nUser's Top 5 Past Purchases (by LogQuantity):")
                if not user_history.empty:
                    for _, row in user_history.head().iterrows():
                         print(f"  - {row['StockCode']}: {row['Description']} (LogQuantity: {row['LogQuantity']:.2f})")
                else:
                    print(f"  User {sample_user_id} has no purchase history in the cleaned dataset.")

                hybrid_recs = get_hybrid_recommendations(
                    user_id=sample_user_id, N_total=10,
                    user_item_df=user_item_df, item_item_cf_similarity_matrix=item_item_cf_similarity_matrix,
                    cf_item_index_map=cf_item_index_map, cf_item_stock_codes=cf_item_stock_codes,
                    cb_cosine_sim_matrix=cb_cosine_sim_matrix, cb_item_index_map=cb_item_index_map,
                    cb_item_stock_codes=cb_item_stock_codes, item_metadata=item_metadata,
                    df_cleaned=df_cleaned, alpha=0.5
                )

                if not hybrid_recs.empty:
                    print("\nTop 10 Hybrid Recommendations:")
                    for i, row in hybrid_recs.reset_index(drop=True).iterrows():
                        print(f"{i+1}. {row['StockCode']} - {row['Description']}")
                        print(f"   Hybrid Score: {row['Hybrid_Score']:.4f} (CF_Norm: {row['CF_Score_Norm']:.2f}, CB: {row['CB_Score']:.2f})")
                else:
                    print(f"Could not generate hybrid recommendations for user {sample_user_id}.")
            else:
                print("Could not select a valid sample user ID for recommendations.")
        else:
            print("User-item matrix for CF is empty or None. Cannot proceed to generate recommendations.")
    else:
        print("Data loading or preprocessing failed. Cannot proceed with recommendations.")

An unexpected error occurred during data loading: name 'full_excel_path' is not defined
Data loading or preprocessing failed. Cannot proceed with recommendations.
